> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [1]:
### Escreva sua resposta aqui
import pandas as pd

df = pd.read_csv('salaries.csv')
print(df.head())
print(df.info())

  FIRST NAME   LAST NAME SEX         DOJ CURRENT DATE DESIGNATION   AGE  \
0     TOMASA       ARMEN   F   5-18-2014   01-07-2016     Analyst  21.0   
1      ANNIE         NaN   F         NaN   01-07-2016   Associate   NaN   
2      OLIVE        ANCY   F   7-28-2014   01-07-2016     Analyst  21.0   
3     CHERRY     AQUILAR   F  04-03-2013   01-07-2016     Analyst  22.0   
4       LEON  ABOULAHOUD   M  11-20-2014   01-07-2016     Analyst   NaN   

   SALARY        UNIT  LEAVES USED  LEAVES REMAINING  RATINGS  PAST EXP  
0   44570     Finance         24.0               6.0      2.0         0  
1   89207         Web          NaN              13.0      NaN         7  
2   40955     Finance         23.0               7.0      3.0         0  
3   45550          IT         22.0               8.0      3.0         0  
4   43161  Operations         27.0               3.0      NaN         3  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2639 entries, 0 to 2638
Data columns (total 13 columns)

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [2]:
### Escreva sua resposta aqui
from enum import Enum
from sqlalchemy import Column, Integer, String, Float, Date, Enum as SqlEnum
from sqlalchemy.orm import declarative_base

class SexEnum(Enum):
    F = "F"
    M = "M"

class DesignationEnum(Enum):
    ANALYST = "Analyst"
    ASSOCIATE = "Associate"
    SENIOR_ANALYST = "Senior Analyst"
    SENIOR_MANAGER = "Senior Manager"
    MANAGER = "Manager"
    DIRECTOR = "Director"

class UnitEnum(Enum):
    FINANCE = "Finance"
    WEB = "Web"
    IT = "IT"
    OPERATIONS = "Operations"
    MARKETING = "Marketing"
    MANAGEMENT = "Management"

Base = declarative_base()

class Employee(Base):
    __tablename__ = 'employees'

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(SqlEnum(SexEnum, native_enum=False))
    doj = Column(Date)
    current_date = Column(Date)
    designation = Column(SqlEnum(DesignationEnum, native_enum=False))
    age = Column(Integer)
    salary = Column(Float)
    unit = Column(SqlEnum(UnitEnum, native_enum=False))
    leaves_used = Column(Integer)
    leaves_remaining = Column(Integer)
    ratings = Column(Float)
    past_exp = Column(Float)

#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [3]:
### Escreva sua resposta aqui
from sqlalchemy import create_engine

engine = create_engine('sqlite:///salarios.db', echo=True)

#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [4]:
### Escreva sua resposta aqui
Base.metadata.create_all(engine)

2025-07-02 19:55:50,330 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-02 19:55:50,331 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employees")
2025-07-02 19:55:50,332 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-02 19:55:50,333 INFO sqlalchemy.engine.Engine COMMIT


#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [5]:
### Escreva sua resposta aqui
df['DOJ'] = pd.to_datetime(df['DOJ'])
df['CURRENT DATE'] = pd.to_datetime(df['CURRENT DATE'])

df.columns = [col.lower().replace(' ', '_') for col in df.columns] # renomeia colunas para bater com os nomes da tabela

df.to_sql('employees', con=engine, if_exists='append', index=False) # insere os dados na tabela

2025-07-02 19:55:50,344 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-02 19:55:50,347 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employees")
2025-07-02 19:55:50,348 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-02 19:55:50,375 INFO sqlalchemy.engine.Engine INSERT INTO employees (first_name, last_name, sex, doj, "current_date", designation, age, salary, unit, leaves_used, leaves_remaining, ratings, past_exp) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2025-07-02 19:55:50,375 INFO sqlalchemy.engine.Engine [generated in 0.02058s] (('TOMASA', 'ARMEN', 'F', '2014-05-18 00:00:00.000000', '2016-01-07 00:00:00.000000', 'Analyst', 21.0, 44570, 'Finance', 24.0, 6.0, 2.0, 0), ('ANNIE', None, 'F', None, '2016-01-07 00:00:00.000000', 'Associate', None, 89207, 'Web', None, 13.0, None, 7), ('OLIVE', 'ANCY', 'F', '2014-07-28 00:00:00.000000', '2016-01-07 00:00:00.000000', 'Analyst', 21.0, 40955, 'Finance', 23.0, 7.0, 3.0, 0), ('CHERRY', 'AQUILAR', 'F', '2013-04-03 00:00:00.

2639

#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [6]:
### Execute aqui sua query SQL com SQLAlchemy
from sqlalchemy import text

with engine.connect() as connection:
    query = text("""
        SELECT designation,
               MIN(salary)/12 AS min_salary_monthly,
               MAX(salary)/12 AS max_salary_monthly,
               AVG(salary)/12 AS avg_salary_monthly
        FROM employees
        GROUP BY designation
        ORDER BY designation
    """)
    result = connection.execute(query)
    
    for row in result:
        print(row)

2025-07-02 19:55:50,392 INFO sqlalchemy.engine.Engine 
        SELECT designation,
               MIN(salary)/12 AS min_salary_monthly,
               MAX(salary)/12 AS max_salary_monthly,
               AVG(salary)/12 AS avg_salary_monthly
        FROM employees
        GROUP BY designation
        ORDER BY designation
    
2025-07-02 19:55:50,393 INFO sqlalchemy.engine.Engine [generated in 0.00071s] ()
('Analyst', 3333.4166666666665, 4165.0, 3751.675987685993)
('Associate', 5846.166666666667, 8300.25, 7266.915094339623)
('Director', 17832.25, 32342.666666666668, 23914.265625)
('Manager', 8343.666666666666, 12407.5, 10522.716049382716)
('Senior Analyst', 4170.333333333333, 5830.5, 4991.778792134832)
('Senior Manager', 12614.416666666666, 16631.416666666668, 14888.689516129032)


In [7]:
### Execute aqui sua query SQL com SQLAlchemy + Pandas
with engine.connect() as connection:
    query = """
        SELECT designation,
               MIN(salary)/12 AS min_salary_monthly,
               MAX(salary)/12 AS max_salary_monthly,
               AVG(salary)/12 AS avg_salary_monthly
        FROM employees
        GROUP BY designation
        ORDER BY designation
    """
    df_grouped = pd.read_sql_query(query, con=connection)

print(df_grouped)

2025-07-02 19:55:50,402 INFO sqlalchemy.engine.Engine 
        SELECT designation,
               MIN(salary)/12 AS min_salary_monthly,
               MAX(salary)/12 AS max_salary_monthly,
               AVG(salary)/12 AS avg_salary_monthly
        FROM employees
        GROUP BY designation
        ORDER BY designation
    
2025-07-02 19:55:50,403 INFO sqlalchemy.engine.Engine [raw sql] ()
      designation  min_salary_monthly  max_salary_monthly  avg_salary_monthly
0         Analyst         3333.416667         4165.000000         3751.675988
1       Associate         5846.166667         8300.250000         7266.915094
2        Director        17832.250000        32342.666667        23914.265625
3         Manager         8343.666667        12407.500000        10522.716049
4  Senior Analyst         4170.333333         5830.500000         4991.778792
5  Senior Manager        12614.416667        16631.416667        14888.689516


In [8]:
### Execute aqui sua query com SQLAlchemy ORM
from sqlalchemy.orm import Session
from sqlalchemy import select, func, cast, String

with Session(engine) as session:
    stmt = (
        select(
            cast(Employee.designation, String).label("designation"),
            (func.min(Employee.salary) / 12).label("min_salary_monthly"),
            (func.max(Employee.salary) / 12).label("max_salary_monthly"),
            (func.avg(Employee.salary) / 12).label("avg_salary_monthly"),
        )
        .group_by(Employee.designation)
        .order_by(Employee.designation)
    )

    result = session.execute(stmt).all()

    for row in result:
        print(row)

2025-07-02 19:55:50,411 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-02 19:55:50,415 INFO sqlalchemy.engine.Engine SELECT CAST(employees.designation AS VARCHAR) AS designation, min(employees.salary) / ? AS min_salary_monthly, max(employees.salary) / ? AS max_salary_monthly, avg(employees.salary) / ? AS avg_salary_monthly 
FROM employees GROUP BY employees.designation ORDER BY employees.designation
2025-07-02 19:55:50,415 INFO sqlalchemy.engine.Engine [generated in 0.00068s] (12, 12, 12)
('Analyst', 3333.4166666666665, 4165.0, 3751.675987685993)
('Associate', 5846.166666666667, 8300.25, 7266.915094339623)
('Director', 17832.25, 32342.666666666668, 23914.265625)
('Manager', 8343.666666666666, 12407.5, 10522.716049382716)
('Senior Analyst', 4170.333333333333, 5830.5, 4991.778792134832)
('Senior Manager', 12614.416666666666, 16631.416666666668, 14888.689516129032)
2025-07-02 19:55:50,418 INFO sqlalchemy.engine.Engine ROLLBACK
